# Домашнее задание 1 (50 баллов)
v4

В этом домашнем задании вы познакомитесь с основами NLP, научитесь обрабатывать тексты.

Будет несколько теоретических вопросов, обозначенных символом 🤔.

Блоки, где необходимо написать свой код, обозначены через YOUR_CODE_HERE. Код, который уже написан, менять не нужно. Блоки в разных частях задания зависят друг от друга, поэтому не вносите больших изменений, чтобы ничего не сломалось.

In [1]:
!pip install gensim nltk torch tqdm seqeval

In [2]:
from typing import List, Dict, Tuple

In [3]:
YOUR_CODE_HERE = None  # заглушка, здесь ничего не трогайте

## Токенизация (5 баллов)

Токенизация - это процесс преобразования текста в набор токенов.
Наивная реализация разбивает текст по пробелам. Более умные реализации учитывают пунктуацию.

Научимся работать с токенизацией NLTK, где уже реализована работа с пунктуацией.

https://www.nltk.org/

In [4]:
import nltk

# https://www.nltk.org/nltk_data/
# nltk.download("punkt")
# nltk.download('punkt_tab')

In [5]:
def tokenize(text: str, language: str = "english", lower: bool = False) -> List[str]:
    # YOUR_CODE_HERE
    if lower:
        text = text.lower()
    return nltk.word_tokenize(text, language=language)

assert tokenize("") == []
assert tokenize("Hello, world!") == ["Hello", ",", "world", "!"]
assert tokenize("EU rejects German call to boycott British lamb.") == ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]

In [6]:
def split_sentences(text: str, language: str = "english", lower: bool = False) -> List[str]:
    # YOUR_CODE_HERE
    if lower:
        text = text.lower()
    return nltk.sent_tokenize(text, language=language)

assert split_sentences("") == []
assert split_sentences("Hello, world!") == ["Hello, world!"]
assert split_sentences("Hello, world! I love Python!") == ["Hello, world!", "I love Python!"]

## Векторизация (5 баллов)

Чтобы работать с текстами, нужно преобразовать токены (слова) в векторы.
Для этого сначала необходимо составить словарь и сопоставить каждому слову уникальный индекс.
В образовательных целях реализуем этот словарь с нуля.

In [7]:
from collections import defaultdict, Counter

In [8]:
class Vocabulary:
    def __init__(self, texts: List[str], language: str = "english", min_count: int = 1, lower: bool = False):
        """
        Создание словаря.

        :param texts: коллекция текстов
        :param language: язык текстов
        :param min_count: минимальная частота слова для попадания в словарь
        """
        self.language = language
        self.lower = lower
        self.min_count = min_count
        self.word2idx = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.idx2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.word2count = Counter()
        self._build_vocabulary(texts)

    def _build_vocabulary(self, texts: List[str]):
        for text in texts:
            # YOUR_CODE_HERE
            tokenized_text = tokenize(text, self.language, self.lower)
            for word in tokenized_text:
                self.word2count[word] += 1

        for word, count in self.word2count.items():
            # YOUR_CODE_HERE
            if count >= self.min_count:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word[idx] = word

    def encode_word(self, text: str) -> int:
        if self.lower:
            text = text.lower()
        return self.word2idx.get(text, self.word2idx["<UNK>"])

    def encode(self, text: str) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :return: набор индексов токенов
        """
        # YOUR_CODE_HERE
        return [self.encode_word(word) for word in tokenize(text, self.language, self.lower)]

    def decode(self, input_ids: List[int]) -> str:
        """
        Декодирование набора индексов в текст.

        :param input_ids: набор индексов токенов
        :return: текст
        """
        return " ".join([self.idx2word[idx] for idx in input_ids if idx != self.word2idx["<PAD>"]])

    def __len__(self):
        return len(self.word2idx)

    def __contains__(self, item):
        return item in self.word2idx

    def __iter__(self):
        return iter(self.word2idx)

    def __str__(self):
        return str(self.word2idx)

In [9]:
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
encoded = vocab.encode("Hello, Python! I love you")
assert vocab.decode(encoded) == "hello , python ! i love <UNK>"

## $n$-граммные языковые модели (10 баллов)

Напишем собственную $n$-граммную модель.

In [10]:
class NGramLanguageModel:
    def __init__(self, n: int, vocabulary: Vocabulary, texts: List[str]):
        """
        Создание n-граммной языковой модели.

        :param n: порядок n-грамм
        :param vocabulary: словарь
        """
        assert n >= 2
        self.n = n
        self.vocabulary = vocabulary
        self.frequencies = defaultdict(lambda: Counter())  # частота n-грамм
        self.frequencies_of_prefixes = Counter()  # сумма частот n-грамм для префиксов
        self._build_model(texts)

    def _build_model(self, texts: List[str]):
        for text in texts:
            tokens = self.vocabulary.encode(text) + [self.vocabulary.word2idx["<EOS>"]]
            for i in range(len(tokens)):
                # YOUR_CODE_HERE
                if not i:
                    prefix = []
                else:
                    prefix = tokens[max(0, i - self.n + 1):i]
                prefix = tuple(prefix)
                current_token = tokens[i]
                self.frequencies[prefix][current_token] += 1
                self.frequencies_of_prefixes[prefix] += 1

    def _get_probability(self, prefix: List[int], token: int) -> float:
        # YOUR_CODE_HERE
        prefix = tuple(prefix)
        return self.frequencies[prefix][token] / self.frequencies_of_prefixes.get(prefix, 1)

    def generate_next_token(self, prefix: List[int]) -> int:
        """
        Генерация следующего токена по префиксу.

        :param prefix: префикс
        :return: следующий токен
        """
        # YOUR_CODE_HERE
        prefix = tuple(prefix)
        distribution = self.frequencies[prefix]
        if not distribution:
            return self.vocabulary.word2idx["<UNK>"]
        next_token, _ = distribution.most_common(1)[0]
        return next_token

    def autocomplete(self, text: str, max_len: int = 32) -> str:
        """
        Автоматическое завершение текста.

        :param text: текст
        :param max_len: максимальная длина текста
        :return: завершенный текст
        """
        tokens = self.vocabulary.encode(text)
        assert tokens

        # YOUR_CODE_HERE
        while len(tokens) < max_len:
            i = len(tokens)
            if not i:
                prefix = []
            else:
                prefix = tokens[max(0, i - self.n + 1):i]
            current_token = self.generate_next_token(prefix)
            tokens.append(current_token)
            if current_token == self.vocabulary.word2idx["<EOS>"]:
                break

        return self.vocabulary.decode(tokens)

In [11]:
texts = ["Hello, world!", "I love Python!", "Hello, Python"]
vocab = Vocabulary(["Hello, world!", "I love Python!"], min_count=1, lower=True)
print(vocab)
ngram_lm = NGramLanguageModel(2, vocab, texts)
assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "hello , python ! <EOS>"

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'hello': 4, ',': 5, 'world': 6, '!': 7, 'i': 8, 'love': 9, 'python': 10}


🤔 Можно ли использовать $n$-граммную языковую модель, когда длина префикса меньше, чем $n-1$? Если да, то как? Если нет, то почему?

Да, можно. Добавим в начало каждого текста $n - 1$ токен <SOS>. Это эквивалентно тому, чтобы в коде использовать префикс меньшей длины (если длина префикса меньше $n - 1$, вы всегда можем мысленно поставить нужное число токенов <SOS> в начале).

🤔 Что произойдет, если в $n$-граммной языковой модели взять достаточно большое $n$?

Матрица подсчетов будет очень разреженной, понадобится много данных, чтобы собрать статистики. Это связано с проклятием размерности.

## Word2Vec (5 баллов)

Познакомимся с моделью Word2Vec. Для русского языка есть обученные модели.
https://rusvectores.org/ru/models/

Модели для английского языка: https://github.com/piskvorky/gensim-data

Научимся доставать эмбеддинги слов из обученных моделей.

In [12]:
from gensim.models import KeyedVectors
import gensim.downloader as api

In [13]:
info = api.info()  # show info about available models/datasets

In [14]:
info

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [15]:
# вы можете выбрать любую модель из gensim, например fasstext
# w2v_model = api.load("glove-twitter-25")
w2v_model = KeyedVectors.load_word2vec_format('data/gensim/glove-twitter-25.gz')

In [16]:
print(w2v_model["potato"])

[-0.41654   -0.56071    0.72333    1.0435     0.0098203  0.46871
  0.93296   -1.2629     0.074417  -0.061837   0.9251    -0.25308
 -2.183     -1.3639    -0.30995   -0.98977    1.6252    -1.0291
 -0.047819   0.64689    0.062647   0.54722   -0.36114    0.15535
  1.0872   ]


In [17]:
print(w2v_model.most_similar(positive=["potato", "burger"]))

[('chicken', 0.9732475280761719), ('cheese', 0.9607888460159302), ('waffle', 0.9480125308036804), ('salad', 0.943723201751709), ('fried', 0.9288325905799866), ('steak', 0.926487147808075), ('pepper', 0.9241527915000916), ('fries', 0.9237129092216492), ('soup', 0.9213549494743347), ('spicy', 0.9193410873413086)]


In [18]:
print(w2v_model.most_similar(positive=["potato", "tomato"]))

[('pepper', 0.9478217959403992), ('garlic', 0.9469695091247559), ('avocado', 0.9457836151123047), ('spicy', 0.9435321092605591), ('chicken', 0.9432903528213501), ('beans', 0.9419039487838745), ('onion', 0.9399765729904175), ('fried', 0.9379871487617493), ('cheese', 0.9373629689216614), ('salad', 0.9371017217636108)]


Реализуйте самостояительно методы $3CosAdd$ и $3CosMul$:

$b^* = \arg \max_{w \in W} \cos (w, a^* - a + b)$

$b^* = \arg \max_{w \in W} \frac{\cos(w, b) \times \cos(w, a^*)}{\cos(w, a)}$

In [19]:
import numpy as np

In [20]:
def three_cos_add(a: str, b: str, a_star: str) -> str:
    # YOUR_CODE_HERE
    matrix = np.array(w2v_model.vectors)
    matrix /= (matrix ** 2).sum(axis=1, keepdims=True) ** 0.5
    
    rhs_vector = np.array(w2v_model[a_star] - w2v_model[a] + w2v_model[b])
    rhs_vector /= (rhs_vector ** 2).sum() ** 0.5

    similarities = matrix @ rhs_vector.reshape(-1, 1)
    b_star_index = similarities.reshape(-1).argmax()
    b_star = w2v_model.index_to_key[b_star_index]
    return b_star

In [21]:
def three_cos_mul(a: str, b: str, a_star: str) -> str:
    # YOUR_CODE_HERE
    matrix = np.array(w2v_model.vectors)
    matrix /= (matrix ** 2).sum(axis=1, keepdims=True) ** 0.5
    
    a_vector = np.array(w2v_model[a])
    a_vector /= (a_vector ** 2).sum() ** 0.5
    
    a_star_vector = np.array(w2v_model[a_star])
    a_star_vector /= (a_star_vector ** 2).sum() ** 0.5

    b_vector = np.array(w2v_model[b])
    b_vector /= (b_vector ** 2).sum() ** 0.5
    
    cosines_w_a, cosines_w_a_star, cosines_w_b = (matrix @ np.stack([a_vector, a_star_vector, b_vector], 1)).T
    similarities = cosines_w_b * cosines_w_a_star / cosines_w_a
    b_star_index = similarities.reshape(-1).argmax()
    b_star = w2v_model.index_to_key[b_star_index]
    return b_star

In [22]:
three_cos_add("man", "woman", "king")

'meets'

In [23]:
%%timeit -r 10 -n 1
three_cos_add("man", "woman", "king")

115 ms ± 4.09 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
three_cos_mul("man", "woman", "king")

'pinkett-smith'

In [25]:
%%timeit -r 10 -n 1
three_cos_mul("man", "woman", "king")

123 ms ± 5.32 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Эмбеддинги и RNN в PyTorch (5 баллов)

Для использования эмбеддингов в нейросетях на torch нужен специальный слой Embedding.
Посмотрим, как он работает.

In [26]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [27]:
emd_dim = 6
embedding_layer = nn.Embedding(len(vocab), emd_dim)

batch_size = 2
seq_len = 5

input_ids = torch.tensor([[1, 2, 3, 4, 5], [1, 5, 6, 0, 0]]).long()
key_padding_mask = input_ids > 0

assert input_ids.shape == (batch_size, seq_len)

embeddings = embedding_layer(input_ids)
print(embeddings)
print(embeddings.shape)
assert embeddings.shape == (batch_size, seq_len, emd_dim)

tensor([[[ 1.0438,  1.3737, -1.1807, -0.8451,  0.4941, -0.1848],
         [ 0.1735, -1.2994, -0.1904, -0.6496, -0.8584,  1.7601],
         [ 0.9538,  0.9900, -0.5806,  1.5981,  1.2591,  0.6463],
         [ 0.6117, -1.3136,  0.0854,  0.0716,  0.2873, -0.0266],
         [-0.2610, -1.2148,  1.1984,  0.8749, -1.8212,  0.8222]],

        [[ 1.0438,  1.3737, -1.1807, -0.8451,  0.4941, -0.1848],
         [-0.2610, -1.2148,  1.1984,  0.8749, -1.8212,  0.8222],
         [-0.6442, -1.1098, -0.5715,  1.3968,  0.2889, -0.9777],
         [ 1.2083,  1.5805, -0.0321, -1.4654,  1.0254,  1.9414],
         [ 1.2083,  1.5805, -0.0321, -1.4654,  1.0254,  1.9414]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 5, 6])


В качестве RNN возьмем двунаправленную LSTM.

In [28]:
hidden_size = 7
n_layers = 3

rnn = nn.LSTM(
    emd_dim, hidden_size,
    num_layers=n_layers,
    bidirectional=True,
    batch_first=True,
    dropout=0.2,
)

# Прочитайте туториал по паддингу в RNN
# https://www.geeksforgeeks.org/how-do-you-handle-sequence-padding-and-packing-in-pytorch-for-rnns/

packed_input = pack_padded_sequence(
    embeddings,
    key_padding_mask.sum(1).cpu().numpy(),
    batch_first=True
)
rnn_outputs, (h, c) = rnn(packed_input)
rnn_outputs, input_sizes = pad_packed_sequence(
    rnn_outputs, batch_first=True
)
assert rnn_outputs.shape == (batch_size, seq_len, hidden_size * 2)

🤔 Почему для рекуррентных сетей нужен специальный паддинг?

Для однонаправленных не очень-то и нужен --- монжо западдить обычным методом и проигнорировать выходы сети на месте паддинга. Проблема возникает, когда начинаем использовать двунаправленные сети. Последовательности выровнены только по левому краю, поэтому и трубуется специальная "упаковка".

## Извлечение именованных сущностей (NER) (20 баллов)

Теперь мы готовы к тому, чтобы создать полную сеть для решения задачи распознавания именованных сущностей через потокенную классификацию.

In [29]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Архитектура и подготовка обучения (10 баллов)

In [31]:
class TokenClassificationModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, hidden_size: int, n_layers: int, n_classes: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(
            embedding_dim, hidden_size,
            num_layers=n_layers,
            dropout=0.2,
            bidirectional=True,
            batch_first=True,
        )
        self.n_classes = n_classes
        self.fc = nn.Linear(hidden_size * 2, n_classes)
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=-100, reduction="none")

    def forward(self, x: torch.Tensor, mask: torch.Tensor):
        # YOUR_CODE_HERE
        embeddings = self.embedding(x)
        packed_input = pack_padded_sequence(
            embeddings,
            mask.sum(1).cpu().numpy(),
            enforce_sorted=False,
            batch_first=True,
        )
        rnn_outputs, (h, c) = self.rnn(packed_input)
        rnn_outputs, input_sizes = pad_packed_sequence(
            rnn_outputs,
            batch_first=True,
        )
        return self.fc(rnn_outputs)

    def training_step(self, batch):
        input_ids = batch["input_ids"]
        mask = batch["mask"]
        labels = batch["labels"]

        # YOUR_CODE_HERE
        logits = self.forward(input_ids, mask)
        loss = self.loss_fn(logits.reshape(-1, self.n_classes), labels.reshape(-1))
        loss = loss.sum() / mask.sum()

        return loss

    def validation_step(self, batch):
        # YOUR_CODE_HERE
        input_ids = batch["input_ids"]
        mask = batch["mask"]
        labels = batch["labels"]
        
        logits = self.forward(input_ids, mask)
        loss = self.loss_fn(logits.reshape(-1, self.n_classes), labels.reshape(-1))
        loss = loss.sum() / mask.sum()

        return loss

    def configure_optimizers(self):
        # YOUR_CODE_HERE
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        return optimizer

In [32]:
def collate_fn(data):
    input_ids = []
    masks = []
    labels = []
    max_len = 0
    for sample in data:
        input_ids.append([vocab.encode_word(token) for token in sample["tokens"]])
        labels.append([label2idx[y] for y in sample["labels"]])
        l = len(input_ids[-1])
        masks.append([1] * l)
        max_len = max(max_len, l)
        
    # YOUR_CODE_HERE
    for i in range(len(input_ids)):
        l = len(input_ids[i])
        input_ids[i].extend([0] * (max_len - l))
        labels[i].extend([-100] * (max_len - l))  # значение, по которому не будет считаться лосс. Указывается как ignore_index
        masks[i].extend([0] * (max_len - l))
    batch = {
        "input_ids": torch.tensor(input_ids, dtype=torch.int64, device=device),
        "mask": torch.tensor(masks, dtype=torch.bool, device=device),
        "labels": torch.tensor(labels, dtype=torch.long, device=device)
    }
    return batch

In [33]:
idx2label = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
label2idx = {label: i for i, label in enumerate(idx2label)}

In [34]:
def read_data(path: str) -> List[Dict[str, List[str]]]:
    samples = []
    with open(path) as f:
        sentences = f.read().split("\n\n")
    for sentence in sentences:
        if "-DOCSTART-" in sentence:
            continue
        tokens = []
        labels = []
        for line in sentence.strip().split("\n"):
            if not line:
                continue
            line = line.split()
            tokens.append(line[0])
            labels.append(line[-1])
        if not tokens:
            continue
        samples.append({"tokens": tokens, "labels": labels})
    return samples

In [35]:
train_data = read_data("data/conll2003/train.txt")
val_data = read_data("data/conll2003/valid.txt")
test_data = read_data("data/conll2003/test.txt")
print(len(train_data), len(val_data), len(test_data))

14041 3250 3453


Прочитайте про схемы тегирования в NER: https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)

🤔 Какая схема тегирования используется в этом датасете?

IOB2.

In [36]:
N_EPOCHS = 10
BATCH_SIZE = 32

In [37]:
train_dl = DataLoader(train_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_data, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False)

In [38]:
vocab = Vocabulary([" ".join(example["tokens"]) for example in train_data], min_count=2, lower=True)

In [39]:
len(vocab)

10946

In [40]:
ner_model = TokenClassificationModel(
    vocab_size=len(vocab),
    embedding_dim=w2v_model.vector_size,
    hidden_size=64,
    n_layers=2,
    n_classes=len(label2idx),
)

In [41]:
# инициализируйте эмбеддинги модели через эмбеддинги word2vec
# YOUR_CODE_HERE
for idx, token in vocab.idx2word.items():
    try:
        w2v_vector = w2v_model[token]
    except:
        print(f"No w2v embedding for word {token}")
        continue
    ner_model.embedding.weight.data[idx] = torch.tensor(w2v_vector)
# заморозьте слой эмбеддингов
# YOUR_CODE_HERE
ner_model.embedding.weight.requires_grad = False

No w2v embedding for word <PAD>
No w2v embedding for word <SOS>
No w2v embedding for word <EOS>
No w2v embedding for word <UNK>
No w2v embedding for word 1996-08-22
No w2v embedding for word zwingmann
No w2v embedding for word sheepmeat
No w2v embedding for word fischler
No w2v embedding for word spleens
No w2v embedding for word spongiform
No w2v embedding for word encephalopathy
No w2v embedding for word scrapie
No w2v embedding for word brain-wasting
No w2v embedding for word 47,600
No w2v embedding for word 4,275
No w2v embedding for word 10
No w2v embedding for word 17,000
No w2v embedding for word 1966
No w2v embedding for word 1967
No w2v embedding for word 16
No w2v embedding for word 1969
No w2v embedding for word 1970
No w2v embedding for word 27
No w2v embedding for word shubei
No w2v embedding for word ...
No w2v embedding for word 14.2
No w2v embedding for word year-earlier
No w2v embedding for word 1996
No w2v embedding for word 2.2
No w2v embedding for word 1995
No w2v e

🤔 Как происходит инициализация эмбеддингов для токенов, отсутствующих в модели word2vec?

Для этих токенов эмбеддинги останутся такими же, как и при инициализации слоя, то есть случайными.

### Обучение и инференс (10 баллов)

In [42]:
ner_model.to(device)
print(ner_model)

TokenClassificationModel(
  (embedding): Embedding(10946, 25)
  (rnn): LSTM(25, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=9, bias=True)
  (loss_fn): CrossEntropyLoss()
)


In [43]:
optimizer = ner_model.configure_optimizers()

In [44]:
for epoch in range(N_EPOCHS):
    ner_model.train()
    train_losses = []
    val_losses = []
    for i, batch in tqdm(enumerate(train_dl)):
        loss = ner_model.training_step(batch)
        train_losses.append(loss.detach().cpu().numpy())
        # YOUR_CODE_HERE
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    ner_model.eval()
    with torch.no_grad():
        for i, batch in tqdm(enumerate(val_dl)):
            loss = ner_model.validation_step(batch)
            val_losses.append(loss.cpu().numpy())
    train_loss = sum(train_losses) / len(train_losses)
    val_loss = sum(val_losses) / len(val_losses)

    print(f"{epoch = }: {train_loss = }, {val_loss = }")
    savepath = f"ner_model_{epoch}ep.bin"
    torch.save(ner_model.state_dict(), savepath)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

epoch = 0: train_loss = 0.31044471864684026, val_loss = 0.2037384451010867


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

epoch = 8: train_loss = 0.07679586689113077, val_loss = 0.17381846451298413


0it [00:00, ?it/s]

0it [00:00, ?it/s]

epoch = 9: train_loss = 0.07447622334139087, val_loss = 0.17013154780593764


In [45]:
def predict(model, test_data) -> List[List[Tuple[str, int, int]]]:
    with torch.no_grad():
        model.eval()
        predictions = []
        for batch in tqdm(test_data):
            input_ids = batch["input_ids"]
            mask = batch["mask"]
            lengths = mask.sum(-1)
            logits = model.forward(input_ids, mask)
            preds = logits.argmax(dim=-1)
            for sample_pred, length in zip(preds, lengths):
                sample_pred = sample_pred[:length]
                spans = []  # class, start, end
                start = -1
                cur_label = ""
                for i in range(len(sample_pred)):
                    pred_label = idx2label[sample_pred[i]]
                    prefix = ""
                    if pred_label != "O":
                        prefix, pred_label = pred_label.split("-")

                    if pred_label == "O":
                        if start != -1:
                            spans.append([cur_label, start, i - 1])
                            start = -1
                            cur_label = ""
                    elif pred_label == cur_label and prefix == "I":
                        pass
                    else:
                        if start != -1:
                            spans.append([cur_label, start, i - 1])
                        start = i
                        cur_label = pred_label
                else:
                    if start != -1:
                        spans.append([cur_label, start, i])
                predictions.append(spans)

        return predictions

In [46]:
predictions = predict(ner_model, test_dl)
for sample, pred in zip(test_data, predictions):
    text = " ".join(sample["tokens"])
    print("*" * 100)
    print(text)

    for tag, start, end in pred:
        print(tag, " ".join(sample["tokens"][start:end + 1]))

  0%|          | 0/108 [00:00<?, ?it/s]

****************************************************************************************************
SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
LOC JAPAN
LOC CHINA
****************************************************************************************************
Nadim Ladki
PER Nadim
****************************************************************************************************
AL-AIN , United Arab Emirates 1996-12-06
LOC AL-AIN
LOC United
****************************************************************************************************
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
LOC Japan
MISC Asian Cup
LOC Syria
****************************************************************************************************
But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
LOC China
***************************

Используя библиотеку https://github.com/chakki-works/seqeval, рассчитайте метрики.

Результаты SotA-решений вы найдете на странице https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003.

In [47]:
# YOUR_CODE_HERE
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

y_true = [example["labels"] for example in test_dl.dataset]
y_pred = []
for example, pred in zip(test_dl.dataset, predictions):
    tags = ["O" for _ in range(len(example["labels"]))]
    for tag, start, end in pred:
        tags[start] = f"B-{tag}"
        for i in range(start + 1, end + 1):
            tags[i] = f"I-{tag}"
    y_pred.append(tags)

report = classification_report(y_true, y_pred, scheme=IOB2)
print(report)

              precision    recall  f1-score   support

         LOC       0.79      0.80      0.80      1668
        MISC       0.67      0.57      0.62       702
         ORG       0.69      0.59      0.63      1661
         PER       0.77      0.69      0.73      1617

   micro avg       0.75      0.68      0.71      5648
   macro avg       0.73      0.66      0.69      5648
weighted avg       0.74      0.68      0.71      5648



### Дополнительное задание (не оценивается)

(Опционально) Попробуйте повысить метрики. Вот несколько идей:

1. Взять более тяжелые эмбеддинги.
2. На замораживать слой эмбеддингов.
3. Использовать MLP после RNN.
4. Умная инициализация весов, дропаут, продвинутый оптимизатор, скедулер и т. д.